Import libraries

In [15]:
import pandas as pd
import numpy as np

Data Cleaning

In [16]:
filepath = r'fifa21 raw data v2.csv'
data = pd.read_csv(filepath)
#Creates copy as to avoid modifying original object
df = data.copy()
#Preview top 5 rows
df.head()

C:\Users\Thus\AppData\Local\Temp\ipykernel_2424\2975306374.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filepath)


,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


In [ ]:
#rename OVA
df.rename(columns={'↓OVA':'OVA'},inplace=True)
#Drop unnecessary columns
columns_to_drop = ['photoUrl','playerUrl']
df = df.drop(columns = columns_to_drop)
#Preview top 5 rows
df.head()

,ID,Player,Nationality,Age,OVA,POT,Club,Contract,Positions,Height,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,Lionel Messi,Argentina,33,93,93,\n\n\n\nFC Barcelona,2004 ~ 2021,"RW, ST, CF",170cm,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,C. Ronaldo dos Santos Aveiro,Portugal,35,92,92,\n\n\n\nJuventus,2018 ~ 2022,"ST, LW",187cm,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,Jan Oblak,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,2014 ~ 2023,GK,188cm,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,Kevin De Bruyne,Belgium,29,91,91,\n\n\n\nManchester City,2015 ~ 2023,"CAM, CM",181cm,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar da Silva Santos Jr.,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,2017 ~ 2022,"LW, CAM",175cm,...,High,Medium,5 ★,91,85,86,94,36,59,595


In [18]:
#Check how many duplicate rows
df.duplicated().sum()

np.int64(0)

In [ ]:
#Drop duplicate rows and then recheck
#df = df.drop_duplicates()
#df.duplicated().sum()

np.int64(0)

In [21]:
#remove white space from club column. Saw quite a bit looking at raw data in csv
df['Club'] = df['Club'].str.strip()
df['Club'].head()

0           FC Barcelona
1               Juventus
2        Atlético Madrid
3        Manchester City
4    Paris Saint-Germain
Name: Club, dtype: object

In [22]:
# Function to convert feet and inches to centimeters
def feet_inch_to_cm(height):
    if isinstance(height, str) and "'" in height and "\"" in height:  # Check if it's in 'ft' 'in' format
        feet, inches = height.split("'")
        inches = inches.replace("\"", "")  # Remove the double quote
        total_cm = (int(feet) * 30.48) + (int(inches) * 2.54)
        return total_cm
    return height  # If already in cm or other format, leave it unchanged
# Clean the 'Height' column, call function
df['Height'] = df['Height'].apply(lambda x: feet_inch_to_cm(x) if 'cm' not in str(x) else float(x.replace('cm', '')))
#rename column
df.rename(columns={'Height' : 'Height_cm'},inplace=True)
df['Height_cm']

0        170.0
1        187.0
2        188.0
3        181.0
4        175.0
         ...  
18974    178.0
18975    175.0
18976    179.0
18977    175.0
18978    188.0
Name: Height_cm, Length: 18979, dtype: float64

In [23]:
def convert_weight_to_kg(value):
    if 'lbs' in value:
        # Convert pounds to kilograms (1 lb = 0.453592 kg)
        weight_in_kg = float(value.replace('lbs', '').strip()) * 0.453592
        return weight_in_kg
    elif 'kg' in value:
        # Extract the value as is for kilograms
        weight_in_kg = float(value.replace('kg', '').strip())
        return weight_in_kg
    else:
        return None  # Handle missing or invalid units
# Apply the function to the Weight column
df['Weight'] = df['Weight'].apply(convert_weight_to_kg)
#rename column
df.rename(columns={'Weight' : 'Weight_kg'},inplace=True)
df['Weight_kg']

0        72.0
1        83.0
2        87.0
3        70.0
4        68.0
         ... 
18974    66.0
18975    65.0
18976    74.0
18977    69.0
18978    75.0
Name: Weight_kg, Length: 18979, dtype: float64

In [24]:
df[['Value', 'Wage', 'Release Clause']].head()

,Value,Wage,Release Clause
0,€103.5M,€560K,€138.4M
1,€63M,€220K,€75.9M
2,€120M,€125K,€159.4M
3,€129M,€370K,€161M
4,€132M,€270K,€166.5M


In [25]:
#use to fix Value, Wage, and Release Clause, converts nvarchar into float
def convert_money(value):
    value = value.replace('€', '').replace(',', '')  # Remove euro symbol and commas
    
    if 'K' in value:
        value = float(value.replace('K', '')) * 1_000  # If value has 'K', multiply by 1,000
    elif 'M' in value:
        value = float(value.replace('M', '')) * 1_000_000  # If value has 'M', multiply by 1,000,000
    else:
        value = float(value)  # Otherwise, just turn it into a float
    
    return value

df[['Value', 'Wage', 'Release Clause']] = df[['Value', 'Wage', 'Release Clause']].apply(lambda col: col.map(convert_money))
df[['Value', 'Wage', 'Release Clause']].head()
df[['Value', 'Wage', 'Release Clause']]

,Value,Wage,Release Clause
0,103500000.0,560000.0,138400000.0
1,63000000.0,220000.0,75900000.0
2,120000000.0,125000.0,159400000.0
3,129000000.0,370000.0,161000000.0
4,132000000.0,270000.0,166500000.0
...,...,...,...
18974,100000.0,1000.0,70000.0
18975,130000.0,500.0,165000.0
18976,120000.0,500.0,131000.0
18977,100000.0,2000.0,88000.0


In [26]:
#Remove star
def star_remover(value):
  value=value.rstrip('★')
  return int(value)

df['W/F']=df['W/F'].apply(star_remover)
df['SM']=df['SM'].apply(star_remover)
df['IR']=df['IR'].apply(star_remover)
df[['IR','SM','W/F']].head()

,IR,SM,W/F
0,5,4,4
1,5,5,4
2,3,1,3
3,4,4,5
4,5,5,5


In [27]:
df.head()

,ID,Player,Nationality,Age,OVA,POT,Club,Contract,Positions,Height_cm,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,Lionel Messi,Argentina,33,93,93,FC Barcelona,2004 ~ 2021,"RW, ST, CF",170.0,...,Medium,Low,5,85,92,91,95,38,65,771
1,20801,C. Ronaldo dos Santos Aveiro,Portugal,35,92,92,Juventus,2018 ~ 2022,"ST, LW",187.0,...,High,Low,5,89,93,81,89,35,77,562
2,200389,Jan Oblak,Slovenia,27,91,93,Atlético Madrid,2014 ~ 2023,GK,188.0,...,Medium,Medium,3,87,92,78,90,52,90,150
3,192985,Kevin De Bruyne,Belgium,29,91,91,Manchester City,2015 ~ 2023,"CAM, CM",181.0,...,High,High,4,76,86,93,88,64,78,207
4,190871,Neymar da Silva Santos Jr.,Brazil,28,91,91,Paris Saint-Germain,2017 ~ 2022,"LW, CAM",175.0,...,High,Medium,5,91,85,86,94,36,59,595


In [28]:
# Create a new CSV file
df.to_csv('V2_cleaned.csv', index=False)